In [14]:
%pip install azure-ai-ml
%pip install azureml-rag[hugging_face]
# mlflow==2.5.0 added support for RetrievalQA in mlflow.langchain
%pip install mlflow>=2.5.0

In [1]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient
from azureml.core import Workspace

try:
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    credential = InteractiveBrowserCredential()

try:
    ml_client = MLClient.from_config(credential)
except Exception as ex:
    print(ex)
    # Enter details of your AML workspace
    subscription_id = "<SUBSCRIPTION_ID>"
    resource_group = "<RESOURCE_GROUP>"
    workspace = "<AML_WORKSPACE_NAME>"
    ml_client = MLClient(credential, subscription_id, resource_group, workspace)
print(ml_client)

ws = Workspace(
    subscription_id=ml_client.subscription_id,
    resource_group=ml_client.resource_group_name,
    workspace_name=ml_client.workspace_name,
)
print(ws)

Found the config file in: C:\Users\smamgain\msft\azureml-examples\sdk\python\generative-ai\rag\notebooks\config.json


MLClient(credential=<azure.identity._credentials.default.DefaultAzureCredential object at 0x0000021FDEC551B0>,
         subscription_id=381b38e9-9840-4719-a5a0-61d9585e1e91,
         resource_group_name=smamgain-rg,
         workspace_name=smamgain-eastus-ws)
Workspace.create(name='smamgain-eastus-ws', subscription_id='381b38e9-9840-4719-a5a0-61d9585e1e91', resource_group='smamgain-rg')


In [2]:
aoai_connection_name = "azure-openai-creds"  # TODO(shivam)
aoai_connection_id = None

In [3]:
from azureml.rag.utils.connections import (
    get_connection_by_name_v2,
    create_connection_v2,
)

try:
    aoai_connection = get_connection_by_name_v2(ws, aoai_connection_name)
except Exception as ex:
    # Create New Connection
    # Modify the details below to match the `Endpoint` and API key of your AOAI resource, these details can be found in Azure Portal
    raise RuntimeError(
        "Have you entered your AOAI resource details below? If so, delete me!"
    )
    target = "<target>"  # example: 'https://<endpoint>.openai.azure.com/'
    key = "<key>"
    apiVersion = "2023-03-15-preview"
    if key == "<key>":
        raise RuntimeError(f"Please provide a valid key for the Azure OpenAI service")
    if target == "<target>":
        raise RuntimeError(
            f"Please provide a valid target for the Azure OpenAI service"
        )
    if apiVersion == "<api_version>":
        raise RuntimeError(
            f"Please provide a valid api-version for the Azure OpenAI service"
        )
    aoai_connection_id = create_connection_v2(
        workspace=ws,
        name=aoai_connection_name,
        category="AzureOpenAI",
        target=target,
        auth_type="ApiKey",
        credentials={"key": key},
        metadata={"ApiType": "azure", "ApiVersion": apiVersion},
    )["id"]

aoai_connection_id = aoai_connection["id"]

In [ ]:
mlindex_dir = "./mlindex"
asset_name = "rustdocs_aoai_faiss_mlindex"

In [36]:
import fsspec

mlindex_data = ml_client.data.get(asset_name, label="latest")
fs, _ = fsspec.core.url_to_fs(mlindex_data.path)
fs.download(mlindex_data.path, mlindex_dir, recursive=True)

In [5]:
from azureml.rag.mlindex import MLIndex

mlindex = MLIndex(mlindex_dir)
retriever = mlindex.as_langchain_retriever()
question = "What makes Rust Enums so awesome?"
[doc.page_content[:20] for doc in retriever.get_relevant_documents(question)]

C:\Users\smamgain\AppData\Local\anaconda3\envs\baker\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['# Defining an Enum\n\n',
 '# Concise Control Fl',
 '# Defining an Enum\n\n',
 '# Storing Lists of V']

'2023-03-15-preview'

In [47]:
from langchain.chains import RetrievalQA
from langchain.llms import AzureOpenAI

llm = AzureOpenAI(
    deployment_name="text-davinci-003",
    model="text-davinci-003",
    max_tokens=1500,
    openai_api_base=aoai_connection["properties"]["target"],
    openai_api_key=aoai_connection["properties"]["credentials"]["key"],
    openai_api_version=aoai_connection["properties"]["metadata"]["ApiVersion"],
)
qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="stuff", retriever=retriever
)
qa.run(question)

' Rust Enums are awesome because they allow you to encode possibilities as a single type. Enums are especially useful when you need to work with values that can only be one of a set of possible variants. They also allow you to define methods on them, making them a powerful tool for creating custom types.'

In [48]:
subscription_id = ml_client.subscription_id
resource_group_name = ml_client.resource_group_name
workspace_name = ml_client.workspace_name

def load_retriever(persist_dir):
    """Function to load the retriever during mlflow's load_model().
    Additionally, it fetches AOAI workspace connection to set OPENAI_API_* environment variables that will be used by LLM. 
    """
    from azureml.rag.mlindex import MLIndex
    from mlflow_code.utils import set_openai_env_vars

    set_openai_env_vars(subscription_id, resource_group_name, workspace_name)

    mlindex = MLIndex(persist_dir)
    retriever = mlindex.as_langchain_retriever()
    return retriever

In [50]:
import os

import mlflow

model_dir = "./mlflow-model"

saved_model = mlflow.langchain.save_model(
    qa,
    model_dir,
    loader_fn=load_retriever,
    persist_dir=mlindex_dir,
    code_paths=["./code/mlflow_code"],
    extra_pip_requirements=[
        "azureml-rag[hugging_face,faiss]",
        "mlflow>=2.5.0",
    ],
)

2023/07/25 14:51:52 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\smamgain\msft\azureml-examples\sdk\python\generative-ai\rag\notebooks\deployment\mlflow-model, flavor: langchain), fall back to return ['langchain==0.0.202']. Set logging level to DEBUG to see the full traceback.


In [38]:
# aoai_connection_id

In [37]:
# model: RetrievalQA = mlflow.pyfunc.load_model(model_dir)
# model.predict([{"query": question}])

In [3]:
import os

import mlflow
import openai
from langchain.chains import RetrievalQA
from langchain.chains.loading import load_chain
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import AzureOpenAI
from langchain.vectorstores.azuresearch import AzureSearch

_SETTING ENVIRONMENT VARIABLES_


In [2]:
# setting openai creds as env vars
os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-03-15-preview"
os.environ["OPENAI_API_BASE"] = "https://<azure-openai-endpoint-name>.openai.azure.com/"
os.environ["OPENAI_API_KEY"] = "<azure-openai-api-key>"

In [12]:
# function to load the embeddings
def load_embeddings():
    embeddings = OpenAIEmbeddings(
        deployment="text-embedding-ada-002",
        model="text-embedding-ada-002",
        chunk_size=1,
    )
    return embeddings


# function to load the vectorstore
def load_vectorstore(embeddings):
    # using azure cognitive search as the vector store
    vectorstore = AzureSearch(
        azure_search_endpoint=os.getenv("VECTOR_STORE_ADDRESS"),
        azure_search_key=os.getenv("VECTOR_STORE_PASSWORD"),
        index_name=os.getenv("INDEX_NAME"),
        embedding_function=embeddings.embed_query,
    )
    return vectorstore

_ADDING TEXTS AND METADATA TO THE VECTORSTORE_


In [13]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

loader = TextLoader("./stateoftheunion.txt", encoding="utf-8")

documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

vector_store = load_vectorstore(load_embeddings())
_ = vector_store.add_documents(documents=docs)

_SAVING THE MODEL_


 The president said that Ketanji Brown Jackson is one of our nation's top legal minds and that she will continue Justice Breyer’s legacy of excellence.


c:\Users\RM\.conda\envs\aml-lc_env\lib\site-packages\_distutils_hack\__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
c:\Users\RM\.conda\envs\aml-lc_env\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


_LOADING THE MODEL_


In [17]:
model: RetrievalQA = mlflow.pyfunc.load_model(local_dir)
query = "What did the president say about Ketanji Brown Jackson"
input_row = {"query": query}
answer = model.predict([input_row])
print(answer)

[" The president said that Ketanji Brown Jackson is one of the nation's top legal minds and that she will continue Justice Breyer's legacy of excellence."]


**_Below is the code to deploy the model in azureml_**


In [18]:
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient

credential = DefaultAzureCredential()

_CREATING AN INSTANCE OF MLCLIENT_


In [28]:
ml_client = None
try:
    ml_client = MLClient.from_config(credential)
except Exception as ex:
    print(ex)
    # Enter details of your AML workspace
    subscription_id = "<SUBSCRIPTION_ID>"
    resource_group = "<RESOURCE_GROUP>"
    workspace = "<AML_WORKSPACE_NAME>"
    ml_client = MLClient(credential, subscription_id, resource_group, workspace)
print(ml_client)

Found the config file in: .\config.json


_REGISTERING THE MODEL_


In [21]:
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
)
from azure.ai.ml.constants import AssetTypes

model_name = "aml-langchain-model"
model_local_path = "./mlflow_model"
model = ml_client.models.create_or_update(
    Model(name=model_name, path=model_local_path, type=AssetTypes.MLFLOW_MODEL)
)


Uploading mlflow_model (0.0 MBs): 100%|##########| 4317/4317 [00:02<00:00, 1697.71it/s]




_SETTING UP ENVIRONMENT VARIABLES FOR OPENAI AND AZURE COGNITIVE SEARCH_


In [22]:
openai_env_vars = {
    "OPENAI_API_TYPE": os.getenv("OPENAI_API_TYPE"),
    "OPENAI_API_VERSION": os.getenv("OPENAI_API_VERSION"),
    "OPENAI_API_BASE": os.getenv("OPENAI_API_BASE") ,
    "OPENAI_API_KEY": os.getenv("OPENAI_API_KEY")
}

acs_env_vars = {
    "VECTOR_STORE_ADDRESS": os.getenv("VECTOR_STORE_ADDRESS"),
    "VECTOR_STORE_PASSWORD": os.getenv("VECTOR_STORE_PASSWORD"),
    "INDEX_NAME": os.getenv("INDEX_NAME"),
}

_CREATING THE ENDPOINT WHERE THE MODEL WILL BE DEPLOYED_


In [23]:
endpoint_name = "aml-langchain-"

import datetime

endpoint_name = endpoint_name + datetime.datetime.now().strftime("%m%d%H%M%f")

# create an online endpoint
endpoint = ManagedOnlineEndpoint(name=endpoint_name, auth_mode="key")
ml_client.online_endpoints.begin_create_or_update(endpoint)

_SETTING UP DEPLOYMENT PARAMETERS_


In [24]:
DEPLOYMENT_NAME = "aml-langchain-deployment"
deployment = ManagedOnlineDeployment(
    name=DEPLOYMENT_NAME,
    endpoint_name=endpoint_name,
    model=model,
    environment_variables={
        **openai_env_vars,
        **acs_env_vars,
    },
    instance_type="Standard_DS3_v2",
    instance_count=1,
)

_DEPLOYING TO THE ENDPOINT_


In [25]:
ml_client.online_deployments.begin_create_or_update(deployment)

Check: endpoint aml-langchain-07191453558955 exists


.......................................................................................................

_UPDATING TRAFFIC TO THE ENDPOINT_


In [26]:
endpoint.traffic = {DEPLOYMENT_NAME: 100}
ml_client.online_endpoints.begin_create_or_update(endpoint)

_TESTING THE RESULTS_


In [27]:
ml_client.online_endpoints.invoke(
    endpoint_name=endpoint_name,
    request_file="request.json",
)

'[" The president said that she is a Circuit Court of Appeals Judge and one of our nation\\u2019s top legal minds who will continue Justice Breyer\\u2019s legacy of excellence."]'